# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 14 2022 9:11AM,250657,10,8763000,Yusen – 3D Matrix,Released
1,Nov 14 2022 9:08AM,250656,20,ATR-I000022,"HVL, LLC dba Atrium Innovations",Released
2,Nov 14 2022 9:01AM,250651,10,0086165617,ISDIN Corporation,Released
3,Nov 14 2022 9:01AM,250651,10,0086165616,ISDIN Corporation,Released
4,Nov 14 2022 9:01AM,250651,10,0086165612,ISDIN Corporation,Released
5,Nov 14 2022 9:01AM,250651,10,0086165703,ISDIN Corporation,Released
6,Nov 14 2022 9:01AM,250651,10,0086165696,ISDIN Corporation,Released
7,Nov 14 2022 9:01AM,250651,10,0086165695,ISDIN Corporation,Released
8,Nov 14 2022 9:01AM,250651,10,0086165701,ISDIN Corporation,Released
9,Nov 14 2022 9:01AM,250651,10,0086165704,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,250652,Released,1
19,250654,Released,48
20,250655,Released,4
21,250656,Released,1
22,250657,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
250652,NaN,1.0
250654,NaN,48.0
250655,NaN,4.0
250656,NaN,1.0
250657,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
250634,0.0,2.0
250635,10.0,50.0
250636,35.0,25.0
250637,26.0,12.0
250638,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
250634,0,2
250635,10,50
250636,35,25
250637,26,12
250638,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,250634,0,2
1,250635,10,50
2,250636,35,25
3,250637,26,12
4,250638,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,250634,,2
1,250635,10,50
2,250636,35,25
3,250637,26,12
4,250638,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations"
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation
66,Nov 14 2022 9:00AM,250654,12,Hush Hush
114,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC"
118,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc."
119,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc"
120,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC"
121,Nov 14 2022 8:40AM,250647,19,ACG North America LLC
122,Nov 14 2022 8:37AM,250646,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,,1
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",,1
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,,64
3,Nov 14 2022 9:00AM,250654,12,Hush Hush,,48
4,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",,4
5,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc.",,1
6,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",,1
7,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",,1
8,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,,1
9,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,1,
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,
3,Nov 14 2022 9:00AM,250654,12,Hush Hush,48,
4,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",4,
5,Nov 14 2022 8:54AM,250652,19,"GCH Granules USA, Inc.",1,
6,Nov 14 2022 8:46AM,250649,19,"Avet Pharmaceuticals Labs, Inc",1,
7,Nov 14 2022 8:44AM,250648,19,"Emersa Waterbox, LLC",1,
8,Nov 14 2022 8:40AM,250647,19,ACG North America LLC,1,
9,Nov 14 2022 8:37AM,250646,19,ACG North America LLC,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,1,
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,
3,Nov 14 2022 9:00AM,250654,12,Hush Hush,48,
4,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,1,NaN
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,NaN
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,NaN
3,Nov 14 2022 9:00AM,250654,12,Hush Hush,48,NaN
4,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",4,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Nov 14 2022 9:11AM,250657,10,Yusen – 3D Matrix,1,0.0
1,Nov 14 2022 9:08AM,250656,20,"HVL, LLC dba Atrium Innovations",1,0.0
2,Nov 14 2022 9:01AM,250651,10,ISDIN Corporation,64,0.0
3,Nov 14 2022 9:00AM,250654,12,Hush Hush,48,0.0
4,Nov 14 2022 8:59AM,250655,10,"Snap Medical Industries, LLC",4,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2255788,164,71.0
12,250654,48,0.0
19,1754528,10,30.0
20,250656,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2255788,164,71.0
1,12,250654,48,0.0
2,19,1754528,10,30.0
3,20,250656,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,164,71.0
1,12,48,0.0
2,19,10,30.0
3,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,164.0
1,12,Released,48.0
2,19,Released,10.0
3,20,Released,1.0
4,10,Executing,71.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20
Status,,,,
Executing,71.0,0.0,30.0,0.0
Released,164.0,48.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20
0,Executing,71.0,0.0,30.0,0.0
1,Released,164.0,48.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20
0,Executing,71.0,0.0,30.0,0.0
1,Released,164.0,48.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()